In [1]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn.functional as F
from optimizer import PruneAdam
from model import LeNet, AlexNet
from utils import regularized_nll_loss, admm_loss, \
    initialize_Z_and_U, update_X, update_Z, update_Z_l1, update_U, \
    print_convergence, print_prune, apply_prune, apply_l1_prune
from torchvision import datasets, transforms, models
from tqdm import tqdm
from Fed import FedAvg
import copy

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--dataset', type=str, default="cifar10", choices=["mnist", "cifar10"],
                    metavar='D', help='training dataset (mnist or cifar10)')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--percent', type=list, default=[0.8, 0.92, 0.991, 0.93],
                    metavar='P', help='pruning percentage (default: 0.8)')
parser.add_argument('--alpha', type=float, default=5e-4, metavar='L',
                    help='l2 norm weight (default: 5e-4)')
parser.add_argument('--rho', type=float, default=1e-2, metavar='R',
                    help='cardinality weight (default: 1e-2)')
parser.add_argument('--l1', default=False, action='store_true',
                    help='prune weights with l1 regularization instead of cardinality')
parser.add_argument('--l2', default=False, action='store_true',
                    help='apply l2 regularization')
parser.add_argument('--num_pre_epochs', type=int, default=3, metavar='P',
                    help='number of epochs to pretrain (default: 3)')
parser.add_argument('--num_epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--num_re_epochs', type=int, default=3, metavar='R',
                    help='number of epochs to retrain (default: 3)')
parser.add_argument('--lr', type=float, default=1e-3, metavar='LR',
                    help='learning rate (default: 1e-2)')
parser.add_argument('--adam_epsilon', type=float, default=1e-8, metavar='E',
                    help='adam epsilon (default: 1e-8)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
parser.add_argument('--num_users', action='store_true', default=1,
                    help='Number of users in network')
parser.add_argument('--model', action='store_true', default='resnet',
                    help='Model to train')
args = parser.parse_args(args=[])

In [3]:
classDict = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

# Define a function to separate CIFAR classes by class index

def get_class_i(x, y, i):
    """
    x: trainset.train_data or testset.test_data
    y: trainset.train_labels or testset.test_labels
    i: class label, a number between 0 to 9
    return: x_i
    """
    # Convert to a numpy array
    y = np.array(y)
    # Locate position of labels that equal to i
    pos_i = np.argwhere(y == i)
    # Convert the result into a 1-D list
    pos_i = list(pos_i[:,0])
    # Collect all data that match the desired label
#     x_i = [x[j] for j in pos_i]
    
    return pos_i

In [4]:
def create_dict(dataset_train):
    
    frac = int(len(dataset_train.data) * 0.05)
    x_reserve = dataset_train.data[:frac]
    y_reserve = dataset_train.targets[:frac]
    x_train = dataset_train.data[frac:]
    y_train = dataset_train.targets[frac:]
    
    reserved = get_class_i(x_reserve, y_reserve, classDict['plane']) \
               +get_class_i(x_reserve, y_reserve, classDict['car']) \
               +get_class_i(x_reserve, y_reserve, classDict['bird']) \
               +get_class_i(x_reserve, y_reserve, classDict['cat']) \
               +get_class_i(x_reserve, y_reserve, classDict['deer']) \
               +get_class_i(x_reserve, y_reserve, classDict['dog']) \
               +get_class_i(x_reserve, y_reserve, classDict['frog']) \
               +get_class_i(x_reserve, y_reserve, classDict['horse']) \
               +get_class_i(x_reserve, y_reserve, classDict['ship']) \
               +get_class_i(x_reserve, y_reserve, classDict['truck'])

    train1 = get_class_i(x_train, y_train, classDict['plane']) \
             +get_class_i(x_train, y_train, classDict['car']) \
             +get_class_i(x_train, y_train, classDict['bird'])

    train2 = get_class_i(x_train, y_train, classDict['cat']) \
             +get_class_i(x_train, y_train, classDict['deer']) \
             +get_class_i(x_train, y_train, classDict['dog'])
    train3 = get_class_i(x_train, y_train, classDict['frog']) \
             +get_class_i(x_train, y_train, classDict['horse']) \
             +get_class_i(x_train, y_train, classDict['ship']) \
             +get_class_i(x_train, y_train, classDict['truck'])
    
    dict_users = {0: set(reserved+train3), 1:set(train1), 2:set(train2)}
    return dict_users

class DatasetSplit(torch.utils.data.Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

In [5]:
def pretrain(args, model, device, train_loader, test_loader, optimizer):
    for epoch in range(args.num_pre_epochs):
        print('Pre epoch: {}'.format(epoch + 1))
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = regularized_nll_loss(args, model, output, target)
            loss.backward()
            optimizer.step()
        test(args, model, device, test_loader)


def train(args, model, device, train_loader, test_loader, optimizer, Z, U, report=False):
    model.train()
    print('Epoch: {}'.format(epoch + 1))
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = admm_loss(args, device, model, Z, U, output, target)
        loss.backward()
        optimizer.step()
    X = update_X(model)
    Z = update_Z_l1(X, U, args) if args.l1 else update_Z(X, U, args)
    U = update_U(U, X, Z)
    print_convergence(model, X, Z)
    test(args, model, device, test_loader, report)


iter = 0
def test(args, model, device, test_loader, report=False):
    model.eval()
    test_loss = 0
    correct = 0
    global iter
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    if report:
        writer.add_scalar('train/loss_gossip_admm4', test_loss, iter)
        writer.add_scalar('valid/accuracy_gossip_admm4', correct / len(test_loader.dataset), iter)
        iter+=1


def retrain(args, model, mask, device, train_loader, test_loader, optimizer):
    for epoch in range(args.num_re_epochs):
        print('Re epoch: {}'.format(epoch + 1))
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.prune_step(mask)

        test(args, model, device, test_loader)

In [6]:
### MAIN

writer = SummaryWriter('../../runs/') 
use_cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

args.percent = [0.8, 0.92, 0.93, 0.94, 0.95, 0.99, 0.99, 0.93]
args.num_pre_epochs = 3
args.num_epochs = 125
args.num_re_epochs = 5
args.num_users = 3
args.dataset = 'cifar10'
args.model = 'alex'
args.l1 = True
args.l2 = False

trainset = datasets.CIFAR10('data', train=True, download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                                                  (0.24703233, 0.24348505, 0.26158768))
                         ]))
dict_users = create_dict(trainset)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, download=True,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.49139968, 0.48215827, 0.44653124),
                                              (0.24703233, 0.24348505, 0.26158768))
                     ])), shuffle=True, batch_size=args.test_batch_size, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
mdlz = dict()
for usr in range(args.num_users):
    train_loader = torch.utils.data.DataLoader(DatasetSplit(trainset, dict_users[usr]), batch_size=args.batch_size, shuffle=True, **kwargs)
    if args.model == 'resnet':
        model = models.resnet18(pretrained=False).to(device)
    else:
        model = LeNet().to(device) if args.dataset == "mnist" else AlexNet().to(device)
    
    optimizer = PruneAdam(model.named_parameters(), lr=args.lr, eps=args.adam_epsilon)
    pretrain(args, model, device, train_loader, test_loader, optimizer)
    Z, U = initialize_Z_and_U(model)
    mdlz[usr] = (model, optimizer, Z, U)
    
for epoch in range(args.num_epochs):
    w_locals=[]
    idxs_users = np.random.choice(range(args.num_users), 2, replace=False)
    for usr in idxs_users:
        report = True if usr == 0 else False
        train_loader = torch.utils.data.DataLoader(DatasetSplit(trainset, dict_users[usr]), batch_size=args.batch_size, shuffle=True, **kwargs)
        train(args, mdlz[usr][0], device, train_loader, test_loader, mdlz[usr][1], mdlz[usr][2], mdlz[usr][3], report=report)
        w = mdlz[usr][0].state_dict()
        w_locals.append(copy.deepcopy(w))

    # update global weights
    w_glob = FedAvg(w_locals)

    for idx in idxs_users:
        # copy weight to net_glob
        mdlz[idx][0].load_state_dict(w_glob)

  0%|          | 0/320 [00:00<?, ?it/s]

Pre epoch: 1


  0%|          | 0/320 [00:00<?, ?it/s]


Test set: Average loss: 1.7212, Accuracy: 3303/10000 (33%)

Pre epoch: 2


  0%|          | 0/320 [00:00<?, ?it/s]

...

normalized norm of (weight - projection)
(features.0.weight): 0.3327
(features.3.weight): 0.5774
(features.6.weight): 0.6327
(features.8.weight): 0.7459
(features.10.weight): 0.8810
(classifier.1.weight): 0.9420
(classifier.4.weight): 0.9720
(classifier.6.weight): 0.7221


  0%|          | 0/223 [00:00<?, ?it/s]


Test set: Average loss: 1.5226, Accuracy: 7072/10000 (71%)

Epoch: 125


100%|██████████| 223/223 [00:20<00:00, 11.01it/s]


normalized norm of (weight - projection)
(features.0.weight): 0.3323
(features.3.weight): 0.5764
(features.6.weight): 0.6373
(features.8.weight): 0.7539
(features.10.weight): 0.8844
(classifier.1.weight): 0.9436
(classifier.4.weight): 0.9758
(classifier.6.weight): 0.7170

Test set: Average loss: 1.3831, Accuracy: 6956/10000 (70%)



In [10]:
### Apply pruning
for usr in range(args.num_users):
    mask = apply_l1_prune(mdlz[usr][0], device, args) if args.l1 else apply_prune(mdlz[usr][0], device, args)
    print_prune(model)
    test(args, mdlz[usr][0], device, test_loader)
    retrain(args, model, mask, device, train_loader, test_loader, optimizer)

Apply Pruning based on percentile
[at weight features.0.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 1728 / 1728

[at weight features.3.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 110592 / 110592

[at weight features.6.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 663552 / 663552

[at weight features.8.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 884736 / 884736

[at weight features.10.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 589824 / 589824

[at weight classifier.1.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 4194304 / 4194304

[at weight classifier.4.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 16777216 / 16777216

[at weight classifier.6.weight]
percentage of pruned: 0.0000%
nonzero parameters after pruning: 40960 / 40960

total nonzero parameters after pruning: 23272266 / 23272266 (0.0000%)


  0%|          | 0/223 [00:00<?, ?it/s]


Test set: Average loss: 3.1654, Accuracy: 1747/10000 (17%)

Re epoch: 1


  0%|          | 0/223 [00:00<?, ?it/s]


Test set: Average loss: 1.7493, Accuracy: 7136/10000 (71%)

Re epoch: 2


...


Test set: Average loss: 1.2641, Accuracy: 6790/10000 (68%)

Re epoch: 4


  0%|          | 0/223 [00:00<?, ?it/s]


Test set: Average loss: 1.4576, Accuracy: 6741/10000 (67%)

Re epoch: 5


100%|██████████| 223/223 [00:07<00:00, 30.11it/s]



Test set: Average loss: 1.6639, Accuracy: 6680/10000 (67%)



# Testing ground

In [8]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

confusion_matrix = torch.zeros(10, 10)
mdlz[0][0].eval()
# testing
test_loss = 0
correct = 0

l = len(test_loader)
for idx, (data, target) in enumerate(test_loader):
    if use_cuda:
        data, target = data.to(device), target.to(device)
    log_probs = mdlz[0][0](data)
    # sum up batch loss
    test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
    # get the index of the max log-probability
    y_pred = log_probs.data.max(1, keepdim=True)[1]
    correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    for t, p in zip(target.data, y_pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

In [9]:
print(confusion_matrix)

tensor([[726.,  11.,  76.,  31.,  22.,   4.,   4.,   9.,  91.,  26.],
        [ 18., 841.,   7.,  10.,   2.,   4.,  13.,   1.,  41.,  63.],
        [ 59.,   5., 608.,  70., 104.,  45.,  52.,  21.,  23.,  13.],
        [ 26.,   8.,  94., 514.,  83., 142.,  52.,  39.,  24.,  18.],
        [ 18.,   2.,  81.,  64., 697.,  33.,  40.,  47.,  15.,   3.],
        [ 12.,   4.,  60., 195.,  63., 568.,  21.,  48.,  18.,  11.],
        [  8.,   5.,  53.,  84.,  50.,  14., 769.,   6.,   9.,   2.],
        [ 23.,   3.,  46.,  42.,  88.,  56.,   8., 707.,   7.,  20.],
        [ 38.,  23.,  11.,  16.,   2.,   9.,   3.,   2., 883.,  13.],
        [ 40., 111.,   7.,  21.,   4.,   0.,   4.,  12.,  45., 756.]])


In [8]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

confusion_matrix = torch.zeros(10, 10)
mdlz[0][0].eval()
# testing
test_loss = 0
correct = 0

l = len(test_loader)
for idx, (data, target) in enumerate(test_loader):
    if use_cuda:
        data, target = data.to(device), target.to(device)
    log_probs = mdlz[0][0](data)
    # sum up batch loss
    test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
    # get the index of the max log-probability
    y_pred = log_probs.data.max(1, keepdim=True)[1]
    correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    for t, p in zip(target.data, y_pred.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

In [9]:
print(confusion_matrix)

tensor([[784.,  14.,  44.,  29.,  11.,   8.,   4.,  10.,  71.,  25.],
        [ 10., 831.,   4.,  14.,   1.,   1.,   9.,   2.,  43.,  85.],
        [ 80.,   6., 630.,  88.,  59.,  45.,  50.,  24.,  11.,   7.],
        [ 31.,  17.,  65., 535.,  46., 172.,  54.,  37.,  28.,  15.],
        [ 35.,   2.,  87.,  93., 625.,  44.,  38.,  65.,   9.,   2.],
        [ 14.,   7.,  51., 205.,  40., 590.,  17.,  49.,  14.,  13.],
        [  8.,   5.,  47.,  97.,  33.,  34., 764.,   3.,   6.,   3.],
        [ 18.,   4.,  27.,  60.,  47.,  70.,   6., 749.,   5.,  14.],
        [ 75.,  20.,   2.,  21.,   5.,   6.,   5.,   4., 844.,  18.],
        [ 32.,  80.,   9.,  28.,   2.,   8.,   3.,  18.,  25., 795.]])
